<a href="https://colab.research.google.com/github/Amarsinh0/PROJECT_WORK/blob/main/2)US_Elections_result_prediction_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [75]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from textblob import TextBlob
from wordcloud import WordCloud
import plotly.graph_objects as go
import plotly.express as px



In [76]:
trump = pd.read_csv("/Trumpall2.csv")


In [77]:
biden = pd.read_csv("/Bidenall2.csv",encoding='latin-1')


In [78]:
print(trump.head(),'n\n\n\n\n')
print(biden.head())

              user                                               text
0      manny_rosen   @sanofi please tell us how many shares the Cr...
1        osi_abdul   https://t.co/atM98CpqF7  Like, comment, RT #P...
2          Patsyrw   Your AG Barr is as useless &amp; corrupt as y...
3  seyedebrahimi_m   Mr. Trump! Wake Up!  Most of the comments bel...
4    James09254677   After 4 years you think you would have figure... n




           user                                               text
0   MarkHodder3  @JoeBiden And weâll find out who won in 2026...
1    K87327961G  @JoeBiden Your Democratic Nazi Party cannot be...
2      OldlaceA                        @JoeBiden So did Lying Barr
3    penblogger  @JoeBiden It's clear you didnt compose this tw...
4  Aquarian0264         @JoeBiden I will vote in person thank you.


# **Sentiment Analysis**

Now, I will get started with sentiment analysis. I will do it by using the Textblob package in Python. Here I will use this package to perform simple text classification in either positive or negative on the basis of sentiment analysis:

In [79]:
textblob1 = TextBlob(trump_reviews["text"][10])
print("Trump :",textblob1.sentiment)

Trump : Sentiment(polarity=0.15, subjectivity=0.3125)


In [80]:
textblob2 = TextBlob(biden["text"][500])
print("Biden :",textblob2.sentiment)


Biden : Sentiment(polarity=0.6, subjectivity=0.9)


In [81]:
from textblob import TextBlob

# we have the 'trump' and 'biden' lists containing text data
trump_sentiments = []

for review in trump:
    sentiment = find_pol(review)
    trump_sentiments.append(sentiment)

biden_sentiments = []

for review in biden:
    sentiment = find_pol(review)
    biden_sentiments.append(sentiment)

# we have the sentiment values in the 'trump_sentiments' and 'biden_sentiments' lists



In [86]:

def find_pol(review):
    return TextBlob(review).sentiment.polarity

trump["Sentiment Polarity"] = trump["text"].apply(find_pol)
print(trump["Sentiment Polarity"])



0       0.050
1       0.000
2      -0.500
3       0.500
4       0.000
        ...  
2783    0.000
2784    0.000
2785    0.225
2786    0.000
2787   -0.500
Name: Sentiment Polarity, Length: 2788, dtype: float64


In [87]:

def find_pol(review):
    return TextBlob(review).sentiment.polarity

biden["Sentiment Polarity"] = biden["text"].apply(find_pol)
print(biden["Sentiment Polarity"])

0       0.00
1       0.00
2       0.00
3       0.05
4       0.00
        ... 
2535    0.15
2536    0.00
2537    0.00
2538    0.10
2539    0.20
Name: Sentiment Polarity, Length: 2540, dtype: float64


**Now before moving forward let’s understand what is Polarity. Polarity ranges from -1 to +1(negative to positive) and tells whether the text has negative sentiments or positive sentiments. Polarity tells about factual information.**

# **Sentiment Polarity on Both the candidates:**

Now I will add a new attribute in both the datasets by the name of “Expression Label”:

In [91]:
trump["Expression Label"] = np.where(trump["Sentiment Polarity"]>0, "positive", "negative")
trump["Expression Label"][trump["Sentiment Polarity"]==0]="Neutral"
print(trump["Expression Label"])



0       positive
1        Neutral
2       negative
3       positive
4        Neutral
          ...   
2783     Neutral
2784     Neutral
2785    positive
2786     Neutral
2787    negative
Name: Expression Label, Length: 2788, dtype: object


<ipython-input-91-4bb9d5ec13d4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trump["Expression Label"][trump["Sentiment Polarity"]==0]="Neutral"


In [93]:
biden["Expression Label"] = np.where(biden["Sentiment Polarity"]>0, "positive", "negative")
biden["Expression Label"][trump["Sentiment Polarity"]==0]="Neutral"
print(biden.head())

           user                                               text  \
0   MarkHodder3  @JoeBiden And weâll find out who won in 2026...   
1    K87327961G  @JoeBiden Your Democratic Nazi Party cannot be...   
2      OldlaceA                        @JoeBiden So did Lying Barr   
3    penblogger  @JoeBiden It's clear you didnt compose this tw...   
4  Aquarian0264         @JoeBiden I will vote in person thank you.   

   Sentiment Polarity Expression Label  
0                0.00         negative  
1                0.00          Neutral  
2                0.00         negative  
3                0.05         positive  
4                0.00          Neutral  


<ipython-input-93-527bb8a4ff46>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  biden["Expression Label"][trump["Sentiment Polarity"]==0]="Neutral"


Now I will drop all the tweets with neutral polarity from both the datasets to balance the data equally. I will also perform some data cleaning operations so that at the can we can easily predict the US Elections:

In [95]:
reviews1 = trump[trump['Sentiment Polarity'] == 0.0000]
print(reviews1.shape)

cond1=trump['Sentiment Polarity'].isin(reviews1['Sentiment Polarity'])
trump.drop(trump[cond1].index, inplace = True)
print(trump.shape)



(0, 4)
(1324, 4)


In [96]:
reviews2 = biden[biden['Sentiment Polarity'] == 0.0000]
print(reviews2.shape)

cond2=biden['Sentiment Polarity'].isin(reviews1['Sentiment Polarity'])
biden.drop(biden[cond2].index, inplace = True)
print(biden.shape)

(0, 4)
(1030, 4)


# **Now, before moving forward we need to balance both the datasets:**

In [98]:
# Donald Trump
np.random.seed(10)
remove_n =324
drop_indices = np.random.choice(trump.index, remove_n, replace=False)
df_subset_trump = trump.drop(drop_indices)
print(df_subset_trump.shape)

(1000, 4)


In [100]:
# Joe Biden
np.random.seed(10)
remove_n =30
drop_indices = np.random.choice(biden.index, remove_n, replace=False)
df_subset_biden = biden.drop(drop_indices)
print(df_subset_biden.shape)

(1000, 4)


# **predict the US Elections, by analyzing  positive and negative sentiments in both the accounts:**

In [107]:
count_1 = df_subset_trump.groupby('Expression Label').count()
print(count_1)

negative_per1 = (count_1['Sentiment Polarity'][0]/1000)*10
positive_per1 = (count_1['Sentiment Polarity'][1]/1000)*100

count_2 = df_subset_biden.groupby('Expression Label').count()
print(count_2)

negative_per2 = (count_2['Sentiment Polarity'][0]/1000)*100
positive_per2 = (count_2['Sentiment Polarity'][1]/1000)*100

Politicians = ['Joe Biden', 'Donald Trump']
lis_pos = [positive_per1, positive_per2]
lis_neg = [negative_per1, negative_per2]

fig = go.Figure(data=[
    go.Bar(name='Positive', x=Politicians, y=lis_pos),
    go.Bar(name='Negative', x=Politicians, y=lis_neg)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

                  user  text  Sentiment Polarity
Expression Label                                
negative           449   449                 449
positive           551   551                 551
                  user  text  Sentiment Polarity
Expression Label                                
Neutral            526   526                 526
negative           179   179                 179
positive           295   295                 295
